# Imports

In [1]:
import pandas as pd
import glob
import numpy as np
import mysql.connector as mysql
import warnings
from mysql.connector import Error

# Ignore Warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# ELT

## Read all the files

In [2]:
ht_path = "High Tension/"
lt_path = "Low Tension/"
ht_files = glob.glob(ht_path + "*.csv")
lt_files = glob.glob(lt_path + "*.csv")
ht_files.sort()
lt_files.sort()

## Files corresponding and its variables
*   0 --> 2018-19
*   1 --> 2019-20
*   2 --> 2020-21
*   3 --> 2021-22
*   4 --> 2022-23







In [3]:
ht = []
lt = []
for i in range(len(ht_files)):
  ht.append(pd.read_csv(ht_files[i]))
  lt.append(pd.read_csv(lt_files[i]))

## Get Index of NULL Values in the data

In [4]:
ht_index = []
lt_index = []
for i in range(len(ht)):
  ht_index.append(np.where(ht[i]['Sl. No.'].isnull() == True)[0])
  lt_index.append(np.where(lt[i]['Sl. No.'].isnull() == True)[0])

## Change adding Bengaluru to end of cities

In [5]:
for i in range(len(ht)):
    ht[i]['Divisions'] = ht[i]['Divisions'] + ", Bengaluru"
    lt[i]['Divisions'] = lt[i]['Divisions'] + ", Bengaluru"

## Changing Sl No so that it division corresponds with its area

In [8]:
for x in range(len(ht)):
    # For ht data
    ht_prev = 0
    ht_mapper = 1
    for i in ht_index[x]:
        ht[x]['Sl. No.'].iloc[ht_prev:i] = ht_mapper
        ht_mapper += 1
        ht_prev = i + 1
        
    # For lt data
    lt_prev = 0
    lt_mapper = 1
    for i in ht_index[x]:
        lt[x]['Sl. No.'].iloc[lt_prev:i] = lt_mapper
        lt_mapper += 1
        lt_prev = i + 1

## Creating new DataFrames to store Null values which are wards in Bengulara

In [6]:
ht_area_data = []
lt_area_data = []
for i in range(len(ht)):
    ht_area_data.append(pd.DataFrame(data=ht[i].iloc[ht_index[i]]))
    lt_area_data.append(pd.DataFrame(data=lt[i].iloc[lt_index[i]]))

## Dropping the null values from the database

In [ ]:
for i in range(len(ht)):
    ht[i].dropna(inplace=True)
    lt[i].dropna(inplace=True)

# Create Database

In [ ]:
try:
    conn = mysql.connect(host='localhost', user='root', password='root')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE bescom")
        print("Connection Successfull !!")
except Error as e:
        print(e)